# COGS 108 - Data Checkpoint

# Names

- Dina Dehaini
- Cody Maheu
- Ryan Jackson
- Lauren Taylor
- Andrew Karafilis Tremaine

<a id='research_question'></a>
# Research Question

Can the wealth of a country determine whether their agricultural exports will be used as food (finalized goods for humans) or feed (raw product for consumption by animals to produce other goods)?

# Dataset(s)

**Dataset Name:** GDP (Current US$) **(GDP)**
 - **Link to Dataset:** https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?end=2019&name_desc=false&start=1960
 - **Number of observations:** 265
 - **Description:** World Bank national accounts data, and OECD National Accounts data files of countries’ GDP.
     - **Timeline:** 1960-2019
     - Data are in current U.S. dollars

**Dataset Name:** Who eats the food we grow? **(FAO)**
 - **Link to Dataset:** https://www.kaggle.com/dorbicycle/world-foodfeed-production
 - **Number of observations:** 21,477
 - **Description:** “The Food and Agriculture Organization of the United Nations provides free access to food and agriculture data for over 245 countries and territories.” (https://www.kaggle.com/dorbicycle/world-foodfeed-production)  The dataset that we are interested in is the Food Balance Sheets. It has the data of a country's food supply during a specified reference period  specifically in our case food/feed.  
     - **Timeline:** 1961-2013
     - Food - refers to the quantity of the food item available as human food during the reference period.
     - Feed - refers to the quantity of the food item available for feeding to the livestock and poultry during the reference period.

**Combining Data**
 - The data will be merged together based on their country code (Area abbreviation in FAO, Country code in GDP). We will examine the data that cannot be merged by hand.



# Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as stats

# GDP dataset
gdp = pd.read_csv("GDP.csv")

# Feed vs Food dataset (it's encoded in CP1252...)
fao = pd.read_csv("FAO.csv", encoding="latin1")

# All valid ISO-3166 country codes (for cleaning the GDP dataset)
with open("country_codes.txt", "r") as fh:
    country_codes = set(line[:-1] for line in fh)

ModuleNotFoundError: No module named 'pandas'

# Data Cleaning

Describe your data cleaning steps here.

- The dataset has been read in from CSV files into our Dataframe objects for both FAO and GDP.
- We removed rows from the GDP dataset with invalid country codes.

- We have begun the steps to begin to understand the data and figure out where our data outliers are coming from. (USA skews it heavily.)
- The FAO dataset has no NaN values but GDP has NaN values for years where a given country did not exist (and some years past 2013 for unknown reasons). We will need to factor this in when merging the datasets.


In [ ]:
# Dumping the columns for reference
gdp.columns

In [ ]:
# Dumping the columns for reference
fao.columns

In [ ]:
# Remove any countries from the GDP dataset that aren't in the FAO dataset.
# Also ensure they're all valid country codes -- there are several
# "total" rows that sum together multiple countries in the GDP dataset.
fao_countries = set(fao["Area Abbreviation"])
valid_countries = fao_countries & country_codes
gdp = gdp[[country in valid_countries for country in gdp["Country Code"]]]

# Year, Country Code, Tons of Food, Tons of Feed, Real GDP
data = pd.DataFrame(columns=["year", "country", "food", "feed", "gdp"])

START = 1992
END = 2013

for year in range(START, END + 1):
    # Convert from thousands of tons to tons
    food_totals = fao[fao["Element"] == "Food"].groupby(["Area Abbreviation"])["Y%d" % year].sum() * 1000
    feed_totals = fao[fao["Element"] == "Feed"].groupby(["Area Abbreviation"])["Y%d" % year].sum() * 1000

    for _, country in gdp.iterrows():
        gdp_for_year = country[str(year)]
        food_for_year = food_totals[country["Country Code"]]

        # Throw out unless we have both GDP data and food export data for the year
        # (feed export data can be 0)
        if not np.isnan(gdp_for_year) and food_for_year > 0:
            data = data.append({
                "year": year,
                "country": country["Country Code"],
                "food": food_for_year,
                "feed": feed_totals[country["Country Code"]],
                "gdp": gdp_for_year
            }, ignore_index=True)

In [ ]:
# Dictionary to convert from country codes to full name
country_names = dict(zip(gdp["Country Code"], gdp["Country Name"]))

# Data Analysis & Results (EDA)

In [ ]:
latest = data[data["year"] == 2013]

In [ ]:
# Show the top and bottom 10 countries by GDP in 2013.
out = latest.sort_values("gdp", ascending=False)
out["country"] = out["country"].apply(lambda x: country_names[x])

In [ ]:
out.head(n=10)

In [ ]:
out.tail(n=10)

In [ ]:
# Show the top and bottom 10 countries by food export in 2013.
out = latest.sort_values("food", ascending=False)
out["country"] = out["country"].apply(lambda x: country_names[x])

In [ ]:
out.head(n=10)

In [ ]:
out.tail(n=10)

In [ ]:
# Show the top and bottom 10 countries by feed export in 2013.
out = latest.sort_values("feed", ascending=False)
out["country"] = out["country"].apply(lambda x: country_names[x])

In [ ]:
out.head(n=10)

In [ ]:
out.tail(n=10)

In [ ]:
#latest = data[data["year"] == 1961]

In [ ]:
# Normal and log-scale plots of the distribution of GDP
# Average is a "10 figure" GDP from reading the log plot
_, (normal, log) = plt.subplots(2)

sns.kdeplot(latest["gdp"], cut=0, ax=normal)
sns.kdeplot(np.log10(latest["gdp"]), cut=0, ax=log)

In [ ]:
# Log-scale plots of the distribution of total feed exported and total food exported in 2013
# Measured in tons
_, (normal, log) = plt.subplots(1,2)

feed = sns.kdeplot(np.log10(latest[latest["feed"] > 0]["feed"]), cut=0, ax=normal)
feed.set_xlim([3,10])
food = sns.kdeplot(np.log10(latest["food"]), cut=0, ax=log)
food.set_xlim([3,10])

In [ ]:
# All countries that exported no feed in 2013
out = latest[latest["feed"] <= 0][:]
out["country"] = out["country"].apply(lambda x: country_names[x])

out

In [ ]:
# Number of countries with data present over time
food = fao[fao["Element"] == "Food"].groupby(["Area Abbreviation"]).sum()
years = [np.count_nonzero(food["Y%d" % i]) for i in range(1961, END + 1)]

sns.lineplot(x=range(1961, END+1), y=years)

In [ ]:
mean_gdp = data.groupby("year")["gdp"].agg("mean")
mean_food = data.groupby("year")["food"].agg("mean")
mean_feed = data.groupby("year")["feed"].agg("mean")

sns.scatterplot(x=mean_gdp, y=mean_feed / (mean_food + mean_feed))

In [ ]:
slope, intercept, *stats = stats.linregress(mean_gdp, mean_feed / (mean_food + mean_feed))
lim = np.array([min(mean_gdp), max(mean_gdp)])
sns.scatterplot(x=mean_gdp, y=mean_feed / (mean_food + mean_feed))
sns.lineplot(x=lim, y=slope * lim + intercept)
print("R: %f, P-value: %f, Err: %f" % tuple(stats))

# Project Proposal (updated)

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 2/11  | 4 PM  | Checkpoint 1 Review | Wrangle data |
| 2/14  | 2 PM  | EDA stuff | start checkpoint 2 |
| 2/17  | 4 PM  | prep checkpoint 2 turnin | turn checkpoint 2 in |
| 2/21  | 2 PM  | talk about data analysis | data analysis |
| 2/24  | 4 PM  | more data analysis, clean up notebook | review notebook |
